In [2]:
# QA pipeline
from transformers import pipeline
question_answer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "which deep learning libraries back 🤗 Transformers?"
result = question_answer(question=question, context=context)
print(f"Answer: '{result}'")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Answer: '{'score': 0.9402405042710598, 'start': 78, 'end': 106, 'answer': 'Jax, PyTorch, and TensorFlow'}'


In [3]:
# Read qa_context.txt file
with open("../data/qa_context.txt", "r") as f:
    long_context = f.read()
question_answer(question=question, context=long_context)

{'score': 0.9396279790053086,
 'start': 1891,
 'end': 1918,
 'answer': 'Jax, PyTorch and TensorFlow'}

In [4]:
# Set up the model from scratch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_checkpoint = "distilbert-base-uncased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 62]) torch.Size([1, 62])


In [ ]:
# Mask out everything except the context tokens
import torch

sequence_ids = inputs.sequence_ids()

# Mast everything except the tokens of the context
mask = [i !=1 for i in sequence_ids]

# Unmask the [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None] # convert from list to tensor and add batch dimension at the beginning

start_logits[mask] = -float("inf")
end_logits[mask] = -float("inf")

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

(tensor([9.8204e-08, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.5015e-06, 5.3210e-06,
         1.4555e-07, 2.4348e-06, 1.7173e-07, 2.5318e-05, 3.8216e-05, 1.3030e-06,
         4.5736e-07, 9.0649e-06, 1.6540e-07, 8.7706e-07, 2.4123e-05, 9.9960e-01,
         1.5769e-06, 3.1931e-05, 1.0236e-06, 2.5909e-07, 9.3109e-07, 2.2342e-07,
         1.1279e-06, 2.2715e-04, 1.5993e-05, 6.7228e-07, 4.5640e-07, 1.1396e-07,
         2.0751e-07, 4.3530e-08, 1.4768e-07, 4.5778e-08, 2.2926e-07, 1.0551e-07,
         8.8707e-08, 9.1643e-09, 9.8670e-09, 7.4200e-08, 1.6864e-08, 4.2613e-08,
         1.5442e-08, 4.4833e-08, 9.5902e-09, 5.7620e-08, 1.7640e-08, 4.7698e-08,
         3.6061e-08, 8.7709e-09, 4.0672e-08, 3.3844e-09, 3.4904e-09, 9.6039e-09,
         1.0284e-08, 0.0000e+00], grad_fn=<SelectBackward0>),
 tensor([1.2290e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.

In [10]:
scores = start_probabilities[:, None] * end_probabilities[None, :]
scores = torch.triu(scores)  # only consider end positions after start positions

In [11]:
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(scores[start_index, end_index])

tensor(0.9945, grad_fn=<SelectBackward0>)


In [12]:
# Convert token indices to character indices
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]
print(f"Answer: '{answer}'")

Answer: 'Jax, PyTorch, and TensorFlow'


In [13]:
# Format result
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index].item()
}
print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': 0.9944560527801514}


In [17]:
# Handling long contexts with a sliding window
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    return_tensors="pt",
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(tokenizer.decode(inputs["input_ids"][0]))

[CLS] which deep learning libraries back [UNK] transformers? [SEP] [UNK] transformers : state of the art nlp [UNK] transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. its aim is to make cutting - edge nlp easier to use for everyone. [UNK] transformers provides apis to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. at the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. why should i use transformers? 1. easy - to - use state - of - the - art models : - high performance on nlu and nlg tasks. - low barrier to entry for educators and practitioners. - few user - facing abstractions with just three classes to learn. - a unified a

In [18]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

torch.Size([2, 384])


In [19]:
outputs = model(**inputs)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


In [20]:
# Apply masking to ignore non-context tokens
sequence_ids = inputs.sequence_ids()
mask = [i !=1 for i in sequence_ids]
mask[0] = False
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))
start_logits[mask] = -float("inf")
end_logits[mask] = -float("inf")

In [23]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [24]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores). argmax().item()
    
    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

[(13, 18, 0.323265016078949), (156, 164, 0.9911271333694458)]


In [25]:
# Retrieve the best answer from all the candidates
for candidate, offset in zip(candidates, offsets):
    start_idx, end_idx, score = candidate
    start_char, _ = offset[start_idx]
    _, end_char = offset[end_idx]
    answer = long_context[start_char:end_char]
    print(f"Answer: '{answer}' (score: {score})")

Answer: 'State of the Art NLP' (score: 0.323265016078949)
Answer: 'Jax, PyTorch and TensorFlow' (score: 0.9911271333694458)


In [26]:
# Tokenizer normalization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))

<class 'tokenizers.Tokenizer'>


In [ ]:
# Note: syntax changed in tokenizers v0.13.3
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [30]:
# Pre-tokenization
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are      you?")

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('how', (7, 10)),
 ('are', (11, 14)),
 ('you', (20, 23)),
 ('?', (23, 24))]

In [31]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
tokenizer_gpt2.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are      you?")

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('Ġhow', (6, 10)),
 ('Ġare', (10, 14)),
 ('ĠĠĠĠĠ', (14, 19)),
 ('Ġyou', (19, 23)),
 ('?', (23, 24))]

In [32]:
tokenizer_t5 = AutoTokenizer.from_pretrained("t5-small")
tokenizer_t5.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are      you?")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[('▁Hello,', (0, 6)),
 ('▁how', (7, 10)),
 ('▁are', (11, 14)),
 ('▁you?', (20, 24))]

In [33]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [34]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

from collections import defaultdict
word_freqs = defaultdict(int)

for text in corpus:
    words_with_ofsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_ofsets]
    for word in new_words:
        word_freqs[word] +=1
        
print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


In [35]:
alphabet = []

for word in word_freqs.keys():
    for char in word:
        if char not in alphabet:
            alphabet.append(char)
alphabet.sort()
print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


In [36]:
vocab = ["<|endoftext|>"] + alphabet.copy()

In [37]:
splits = {word: [c for c in word] for word in word_freqs.keys()}
print(splits)

{'This': ['T', 'h', 'i', 's'], 'Ġis': ['Ġ', 'i', 's'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'], 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'], 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'], '.': ['.'], 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'], 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'], 'Ġtokenization': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'], 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'], 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'], 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'], 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'], 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'], 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'], ',': [','], 'Ġyou': ['Ġ', 'y', 'o', 'u'], 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'], 'Ġbe': ['Ġ', 'b', 'e'], 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'], 'Ġto': ['Ġ', 't', 'o'], 'Ġunderstand': ['Ġ', 'u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd'], 'Ġh

In [38]:
# Helper function to compute word pair frequencies
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split)-1):
            pair = (split[i], split[i+1])
            pair_freqs[pair] += freq
    return pair_freqs

In [39]:
pair_freqs = compute_pair_freqs(splits)
for i, key in enumerate(pair_freqs.keys()):
    if i >= 5:
        break
    print(f"{key}: {pair_freqs[key]}")

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7


In [40]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or freq > max_freq:
        best_pair = pair
        max_freq = freq
        
print(f"Best pair: {best_pair} with frequency {max_freq}")

Best pair: ('Ġ', 't') with frequency 7


In [42]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'Ġt']


In [43]:
# A helper function to merge tokens based on the merges dictionary
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) -1:
            if split[i] == a and split[i+1] == b:
                split = split[:i] + [merges[(a, b)]] + split[i+2:]
            else:
                i +=1
        splits[word] = split
    return splits

In [44]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


In [47]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or freq > max_freq:
            best_pair = pair
            max_freq = freq
    
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])
    splits = merge_pair(*best_pair, splits)

In [48]:
print(merges)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab'}


In [49]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab']


In [50]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [51]:
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']